In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
datasnaek_chess_path = kagglehub.dataset_download('datasnaek/chess')

print('Data source import complete.')


Data source import complete.


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
# !conda install -c rapidsai -c nvidia -c conda-forge -c defaults cuml=23.2 python=3.10 cudatoolkit=11.8 -y
# !pip install xgboost --upgrade
from xgboost import XGBRegressor

print('Data source import complete.')


Data source import complete.


In [3]:
import kagglehub
import numpy as np
import xgboost as xgb
import cupy as cp
import cudf as cd
import datetime
import optuna
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from cuml.preprocessing import StandardScaler
from cuml.preprocessing import LabelEncoder
from cuml.ensemble import RandomForestRegressor
from cuml.model_selection import train_test_split
from cuml.metrics import mean_squared_error, r2_score
# Download latest version

data_path = "/kaggle/input/chess/games.csv"
print(data_path)

/kaggle/input/chess/games.csv


In [4]:
df = cd.read_csv(data_path)

In [5]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   id              20058 non-null  object
 1   rated           20058 non-null  bool
 2   created_at      20058 non-null  float64
 3   last_move_at    20058 non-null  float64
 4   turns           20058 non-null  int64
 5   victory_status  20058 non-null  object
 6   winner          20058 non-null  object
 7   increment_code  20058 non-null  object
 8   white_id        20058 non-null  object
 9   white_rating    20058 non-null  int64
 10  black_id        20058 non-null  object
 11  black_rating    20058 non-null  int64
 12  moves           20058 non-null  object
 13  opening_eco     20058 non-null  object
 14  opening_name    20058 non-null  object
 15  opening_ply     20058 non-null  int64
dtypes: bool(1), float64(2), int64(4), object(9)
memory usage: 7.9+ MB


# data

In [6]:
df.drop_duplicates().head()

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


In [7]:
df.isnull().sum()

id                0
rated             0
created_at        0
last_move_at      0
turns             0
victory_status    0
winner            0
increment_code    0
white_id          0
white_rating      0
black_id          0
black_rating      0
moves             0
opening_eco       0
opening_name      0
opening_ply       0
dtype: int64

In [8]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   id              20058 non-null  object
 1   rated           20058 non-null  bool
 2   created_at      20058 non-null  float64
 3   last_move_at    20058 non-null  float64
 4   turns           20058 non-null  int64
 5   victory_status  20058 non-null  object
 6   winner          20058 non-null  object
 7   increment_code  20058 non-null  object
 8   white_id        20058 non-null  object
 9   white_rating    20058 non-null  int64
 10  black_id        20058 non-null  object
 11  black_rating    20058 non-null  int64
 12  moves           20058 non-null  object
 13  opening_eco     20058 non-null  object
 14  opening_name    20058 non-null  object
 15  opening_ply     20058 non-null  int64
dtypes: bool(1), float64(2), int64(4), object(9)
memory usage: 7.9+ MB


In [9]:
df = df.astype({col: 'int' for col in df.select_dtypes(bool).columns})
df['rated'].head(5)

0    0
1    1
2    1
3    1
4    1
Name: rated, dtype: int64

In [10]:
for i in df.select_dtypes(include=cp.number).columns:
    skewness = df[i].skew()
    print(f"Skewness {i} adalah {skewness}")

Skewness rated adalah -1.543073848714904
Skewness created_at adalah -1.7825585131272512
Skewness last_move_at adalah -1.782553530651669
Skewness turns adalah 0.8972837714383513
Skewness white_rating adalah 0.30076617793518823
Skewness black_rating adalah 0.25851032951990566
Skewness opening_ply adalah 1.334556864262132


In [11]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   id              20058 non-null  object
 1   rated           20058 non-null  int64
 2   created_at      20058 non-null  float64
 3   last_move_at    20058 non-null  float64
 4   turns           20058 non-null  int64
 5   victory_status  20058 non-null  object
 6   winner          20058 non-null  object
 7   increment_code  20058 non-null  object
 8   white_id        20058 non-null  object
 9   white_rating    20058 non-null  int64
 10  black_id        20058 non-null  object
 11  black_rating    20058 non-null  int64
 12  moves           20058 non-null  object
 13  opening_eco     20058 non-null  object
 14  opening_name    20058 non-null  object
 15  opening_ply     20058 non-null  int64
dtypes: float64(2), int64(5), object(9)
memory usage: 8.0+ MB


# Feature Enginering

In [12]:
df['created_at'] = cd.to_datetime(df['created_at'], unit='s')
df['last_move_at'] = cd.to_datetime(df['last_move_at'], unit='s')

In [13]:
df['game_duration'] = (df['last_move_at'] - df['created_at']).dt.total_seconds()

In [14]:
df['hour_of_day'] = df['created_at'].dt.hour
df['day_of_week'] = df['created_at'].dt.day_name()

In [15]:
df['rating_diff'] = df['white_rating'] - df['black_rating']
df['avg_rating'] = (df['white_rating'] + df['black_rating'])/2

In [16]:
df['base_time'] = df['increment_code'].str.extract(r'^(\d+)\+').astype('int')
df['increment_time'] = df['increment_code'].str.extract(r'\+(\d+)$').astype('int')

In [17]:
df['moves_list'] = df['moves'].str.split()
df['num_moves'] = df['moves_list'].list.len()

In [18]:
df['victory_status'].unique()

0    outoftime
1       resign
2         mate
3         draw
Name: victory_status, dtype: object

In [19]:
df['winner'] = df['winner'].map({'white': 0, 'black': 1, 'draw': 2})
df['victory_status'] = df['victory_status'].map({'outoftime': 0, 'resign': 1, 'mate': 2, 'draw': 3})

In [20]:
df['opening_eco_encode'] = cd.factorize(df['opening_eco'])[0]
df['opening_name_encode'] = cd.factorize(df['opening_name'])[0]

In [21]:
df.drop(columns=['created_at', 'last_move_at', 'moves', 'opening_eco', 'opening_name', 'increment_code'], inplace=True)

In [22]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   id                   20058 non-null  object
 1   rated                20058 non-null  int64
 2   turns                20058 non-null  int64
 3   victory_status       20058 non-null  int64
 4   winner               20058 non-null  int64
 5   white_id             20058 non-null  object
 6   white_rating         20058 non-null  int64
 7   black_id             20058 non-null  object
 8   black_rating         20058 non-null  int64
 9   opening_ply          20058 non-null  int64
 10  game_duration        20058 non-null  float64
 11  hour_of_day          20058 non-null  int16
 12  day_of_week          20058 non-null  object
 13  rating_diff          20058 non-null  int64
 14  avg_rating           20058 non-null  float64
 15  base_time            20058 non-null  int64
 16  increment_ti

In [23]:
df['is_weekend'] = df['day_of_week'].isin(['Saturday', 'Sunday']).astype(int)
df['is_night_game'] = (df['hour_of_day'] >= 20).astype(int)

In [24]:
df['white_rating_category'] = (
    (df['white_rating'] <= 1200) * 0 +
    ((df['white_rating'] > 1200) & (df['white_rating'] <= 1600)) * 1 +
    ((df['white_rating'] > 1600) & (df['white_rating'] <= 2000)) * 2 +
    (df['white_rating'] > 2000) * 3
)

df['black_rating_category'] = (
    (df['black_rating'] <= 1200) * 0 +
    ((df['black_rating'] > 1200) & (df['black_rating'] <= 1600)) * 1 +
    ((df['black_rating'] > 1600) & (df['black_rating'] <= 2000)) * 2 +
    (df['black_rating'] > 2000) * 3
)

In [25]:
df['first_move_white'] = df['moves_list'].list.get(0)
df['first_move_black'] = df['moves_list'].list.get(1)

df.drop(columns=['moves_list'], inplace=True)

In [26]:
df.select_dtypes(exclude=np.number).info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   id                20058 non-null  object
 1   white_id          20058 non-null  object
 2   black_id          20058 non-null  object
 3   day_of_week       20058 non-null  object
 4   first_move_white  20058 non-null  object
 5   first_move_black  20040 non-null  object
dtypes: object(6)
memory usage: 1.2+ MB


In [27]:
labelencoder = LabelEncoder()
df['day_of_week'] = labelencoder.fit_transform(df['day_of_week'])

In [28]:
mean_turns = df.groupby('white_id')['turns'].mean()
df['white_id'] = df['white_id'].map(mean_turns)

mean_turns = df.groupby('black_id')['turns'].mean()
df['black_id'] = df['black_id'].map(mean_turns)

In [29]:
mean_turns_white = df.groupby('first_move_white')['turns'].mean()
df['first_move_white'] = df['first_move_white'].map(mean_turns_white)

mean_turns_black = df.groupby('first_move_black')['turns'].mean()
df['first_move_black'] = df['first_move_black'].map(mean_turns_black)


In [30]:
mean_turns_black = df.groupby('id')['turns'].mean()
df['id'] = df['id'].map(mean_turns_black)

In [31]:
# df.drop(columns=[''], inplace=True)


In [32]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   id                     20058 non-null  float64
 1   rated                  20058 non-null  int64
 2   turns                  20058 non-null  int64
 3   victory_status         20058 non-null  int64
 4   winner                 20058 non-null  int64
 5   white_id               20058 non-null  float64
 6   white_rating           20058 non-null  int64
 7   black_id               20058 non-null  float64
 8   black_rating           20058 non-null  int64
 9   opening_ply            20058 non-null  int64
 10  game_duration          20058 non-null  float64
 11  hour_of_day            20058 non-null  int16
 12  day_of_week            20058 non-null  uint8
 13  rating_diff            20058 non-null  int64
 14  avg_rating             20058 non-null  float64
 15  base_time              2

# modeling

In [33]:
X = df.drop(columns=['turns'])
y = df['turns']

In [34]:
print(X.isnull().sum())
print(y.isnull().sum())

X = X.dropna()
y = y.loc[X.index] 

id                        0
rated                     0
victory_status            0
winner                    0
white_id                  0
white_rating              0
black_id                  0
black_rating              0
opening_ply               0
game_duration             0
hour_of_day               0
day_of_week               0
rating_diff               0
avg_rating                0
base_time                 0
increment_time            0
num_moves                 0
opening_eco_encode        0
opening_name_encode       0
is_weekend                0
is_night_game             0
white_rating_category     0
black_rating_category     0
first_move_white          0
first_move_black         18
dtype: int64
0


In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fungsi untuk training dan evaluasi model
def objective(trial):
    # Parameter yang akan dioptimasi
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    max_features = trial.suggest_float('max_features', 0.5, 1.0)
    
    # Model Random Forest Regressor dengan GPU
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features=max_features,
        random_state=42
    )
    
    # Training model
    model.fit(X_train_scaled, y_train)
    
    # Evaluasi menggunakan Mean Squared Error (MSE)
    y_pred = model.predict(X_test_scaled)
    
    # Konversi y_test dan y_pred ke CuPy array
    y_test_cupy = y_test.to_cupy()
    y_pred_cupy = cp.array(y_pred)
    
    # Hitung MSE dengan CuPy
    mse = cp.mean((y_test_cupy - y_pred_cupy) ** 2)
    
    return mse

# Optuna Study untuk mencari parameter terbaik
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# Parameter terbaik
best_params = study.best_params
print("Best Parameters:", best_params)

# Train model dengan parameter terbaik
best_model = RandomForestRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    max_features=best_params['max_features'],
    random_state=42,
    n_streams=1
)
best_model.fit(X_train_scaled, y_train)

# Evaluasi akhir
y_pred = best_model.predict(X_test_scaled)

# Konversi y_test dan y_pred ke CuPy array
y_test_cupy = y_test.to_cupy()  # Konversi y_test ke CuPy
y_pred_cupy = cp.array(y_pred)  # Konversi y_pred ke CuPy

# Hitung Mean Squared Error dengan CuPy
mse = cp.mean((y_test_cupy - y_pred_cupy) ** 2)
print("Final MSE:", mse)

[I 2025-04-23 02:21:21,786] A new study created in memory with name: no-name-8c145982-e662-4408-800b-901508d862e1
/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:368: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return init_func(self, *args, **kwargs)
[I 2025-04-23 02:21:22,751] Trial 0 finished with value: 26.33969970935992 and parameters: {'n_estimators': 163, 'max_depth': 3, 'max_features': 0.818827926949842}. Best is trial 0 with value: 26.33969970935992.
[I 2025-04-23 02:21:23,293] Trial 1 finished with value: 5.018068586076645 and parameters: {'n_estimators': 112, 'max_depth': 13, 'max_features': 0.6550937859914303}. Best is trial 1 with value: 5.018068586076645.
[I 2025-04-23 02:21:23,720] Trial 2 finished with value: 8.285507734303433

Best Parameters: {'n_estimators': 30, 'max_depth': 11, 'max_features': 0.6692350343827249}
Final MSE: 3.89298590147677


In [36]:
def mean_absolute_percentage_error(y_true, y_pred):
  y_true, y_pred = cp.array(y_true), cp.array(y_pred)
  return cp.mean(np.abs((y_true - y_pred)/y_true)) * 100

In [37]:
def predict_perform(X_train, X_test, y_train, y_test):
    y_train_pred = best_model.predict(X_train)
    y_test_pred = best_model.predict(X_test)
    
    r2_train = r2_score(y_train.to_cupy(), y_train_pred.to_cupy())
    r2_test = r2_score(y_test.to_cupy(), y_test_pred.to_cupy())
    MAPE = mean_absolute_percentage_error(y_test, y_test_pred)
    
    print(f"🔹 R² (Train): {r2_train:.4f}")
    print(f"🔹 R² (Test) : {r2_test:.4f}")
    print(f"🔹 MAPE     : {MAPE:.3f}%")


predict_perform(X_train_scaled, X_test_scaled, y_train, y_test)

🔹 R² (Train): 0.9994
🔹 R² (Test) : 0.9965
🔹 MAPE     : 0.526%
